Features: Pixelization
======================

A pixelization reconstructs the galaxy's light using a pixel-grid, which is regularized using a prior that forces
the solution to have a degree of smoothness.

This script fits a galaxy model which uses a pixelization to reconstruct the galaxy's light.

A rectangular mesh and constant regularization scheme are used, which are the simplest forms of mesh and
regularization and provide computationally fast and accurate solutions.

For simplicity, the galaxy’s light is modeled using only a pixelization. It is straightforward to include additional
parametric light components in the model.

Pixelizations are covered in detail in chapter 4 of the **HowToGalaxy** lectures.

__JAX GPU Run Times__

Throughout the workspace, it has been emphasised that pixelized galaxy reconstructions are computed using GPU or CPU
via JAX, where the linear algebra fully exploits sparsity in a way which minimizes VRAM use. This example uses
this functionality, and therefore is suitable for datasets with a low number of visibilities (e.g. < 10000) or
many visibilities (E.g. tens of millions).

This example fits the dataset with 273 visibilities used throughout the workspace, so the fit runs in seconds, but
provided the sparse operator formalism is set up correctly, the same code can be used to fit datasets with millions of
visibilities.

If your dataset contains many visibilities (e.g. millions), setting up the matrices for pixelized galaxy reconstruction
which speed up the linear algebra may take tens of minutes, or hours. Once you are comfortable with the API introduced
in this example, the `feature/pixelization/many_visibilities_preparation` explains how this initial setup can be
performed before galaxy modeling and saved to hard disk for fast loading before the model fit.

This script's default setup uses an adaptive 20 x 20 rectangular mesh (400 pixels), which is relatively low resolution
and may not provide the most accurate galaxy modeling results. The mesh resolution can be increased to improve
the fit, and the sparse operator formalism means this should still run fine on my laptop GPUs, requiring less than 4 GB VRAm.

CPU run times are also fast using the sparse operator formalism.

__Contents__

**Advantages & Disadvantages:** Benefits and drawbacks of using an MGE.
**Positive Only Solver:** How a positive solution to the light profile intensities is ensured.
**Dataset & Mask:** Standard set up of interferometer dataset that is fitted.
**Mesh Shape**: Defining the shape of the mesh that reconstructs the source in advance, such that JAX knows static array shapes.
**Pixelization:** How to create a pixelization, including a description of its inputs.
**Fit:** Perform a fit to a dataset using a pixelization, and visualize its results.
**Interpolated Source:** Interpolate the reconstruction from an irregular Voronoi mesh to a uniform square grid and output to a .fits file.
**Reconstruction CSV:** Output the reconstruction to a .csv file, which can be used to perform calculations on the reconstruction.
**Result (Advanced):** API for various pixelization outputs which requires some polishing.
**Simulate (Advanced):** Simulating an interferometer dataset with the inferred pixelized galaxy.

__Advantages__

Many galaxies exhibit complex, asymmetric, and irregular morphologies. Such structures cannot be well approximated by
analytic light profiles such as a Sérsic profile, or even combinations of multiple Sérsic components. Pixelizations are
therefore required to accurately reconstruct this irregular galaxy light.

Even alternative basis-function approaches, such as shapelets or multi-Gaussian expansions, struggle to accurately
reconstruct galaxies with highly complex morphologies or multiple distinct components.

Pixelized galaxy models are also essential for robustly constraining detailed components of a galaxy’s light
distribution. By fitting all of the galaxy light, they reduce degeneracies between different components of the model.

Finally, many science applications aim to study the galaxy itself in detail, in order to learn about distant and
intrinsically faint galaxies. Pixelizations reconstruct the intrinsic galaxy emission, enabling detailed studies of
galaxy structure.

For CCD imaging, a disadvantage of pixelized galaxy reconstructions is they are the most computationally expensive
modeling approach. However, for interferometer datasets, the way that JAX and GPUs can exploit the sparsity in the
linear algebra means pixelized galaxy reconstructions are both significantly faster than other approaches (E.g.
light profiles) and can scale to millions of visibilities.

__Disadvantages__

Galaxy modeling with pixelizations is conceptually more complex. There are additional failure modes, such as
solutions where the galaxy is reconstructed in an unphysical configuration. These issues are discussed in detail
later in the workspace.

As a result, learning to successfully fit galaxy models with pixelizations typically requires more time and experience
than the simpler modeling approaches introduced elsewhere in the workspace.

__Positive Only Solver__

Many codes which use linear algebra typically rely on a linear algabra solver which allows for positive and negative
values of the solution (e.g. `np.linalg.solve`), because they are computationally fast.

This could be problematic, as it means that negative surface brightnesses values can be computed to represent a galaxy's
light, which is clearly unphysical. For a pixelizaiton, this often produces negative pixels which over-fit
the data, producing unphysical solutions.

For CCD imaging datsets pixelized galaxy reconstructions use a positive-only solver, meaning that every pixel
is only allowed to reconstruct positive flux values. This ensures that the reconstruction is physical and
that we don't reconstruct negative flux values that don't exist in the real galaxy.

However, for interferometer datasets this positive-only solver is often disabled, because negative pixel values
can be observed from the measurement process. All interferometer examples therefore disable the positive only solver,
but you may want to consider if using the positive-only solver is appropriate for your dataset.

In [ ]:

from autoconf import jax_wrapper  # Sets JAX environment before other imports

%matplotlib inline
from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

from pathlib import Path

import autogalaxy as ag
import autogalaxy.plot as aplt

__Mask__

We define the ‘real_space_mask’ which defines the grid the image is evaluated using.

In [ ]:
mask_radius = 3.5

real_space_mask = ag.Mask2D.circular(
    shape_native=(256, 256),
    pixel_scales=0.1,
    radius=mask_radius,
)

__Dataset__

Load and plot the `Interferometer` dataset `simple` from .fits files, which we will fit
with the galaxy model.

This includes the method used to Fourier transform the real-space image to the uv-plane and compare
directly to the visiblities. We use a non-uniform fast Fourier transform, which is the most efficient method for
interferometer datasets containing ~1-10 million visibilities.

If you want to use the high resolution ALMA dataset, uncomment the relevant lines of code below after downloading
the data from the repository described in the "High Resolution Dataset" section above.

In [ ]:
dataset_name = "simple"
dataset_path = Path("dataset") / "interferometer" / dataset_name

dataset = ag.Interferometer.from_fits(
    data_path=dataset_path / "data.fits",
    noise_map_path=dataset_path / "noise_map.fits",
    uv_wavelengths_path=dataset_path / "uv_wavelengths.fits",
    real_space_mask=real_space_mask,
    transformer_class=ag.TransformerDFT,
)

dataset_plotter = aplt.InterferometerPlotter(dataset=dataset)
dataset_plotter.subplot_dataset()
dataset_plotter.subplot_dirty_images()

__Sparse Operators__

Pixelized galaxy modeling requires dense linear algebra operations. These calculations are greatly accelerated
using an alternative mathematical approach called the **sparse linear algebra formalism**.

You do not need to understand the full details of the method, but the key point is:

- It exploits the **sparsity** of the matrices used in pixelized source reconstruction.
- This leads to a **significant speed-up on GPU or CPU**, using JAX to perform the linear algebra calculations.

To enable this feature, we call `apply_sparse_operator()` on the dataset. This computes and stores a NUFFT operator 
matrix.

For datasets with over 100000 visibilities and many pixels in their real-space mask, this computation
can take 10 minutes or hours (for the small dataset loaded above its miliseconds). The `show_progress` input outputs
a progress bar to the terminal so you can monitor the computation, which is useful when it is slow

When computing it is slow, it is recommend you compute it once, save it to hard-disk, and load it
before modeling. The example `pixelization/many_visibilities_preparation.py` illustrates how to do this.

In [ ]:
dataset = dataset.apply_sparse_operator(use_jax=True, show_progress=True)

__Settings__

As discussed above, disable the default position only linear algebra solver so the
reconstruction can have negative pixel values.

In [ ]:
settings = ag.Settings(use_positive_only_solver=False)

__Over Sampling__

If you are familiar with using imaging data, you may have seen that a numerical technique called over sampling is used,
which evaluates light profiles on a higher resolution grid than the image data to ensure the calculation is accurate.

Interferometer does not observe galaxies in a way where over sampling is necessary, therefore all interferometer
calculations are performed without over sampling.

__Mesh Shape__

The `mesh_shape` parameter defines number of pixels used by the rectangular mesh to reconstruct the source,
set below to 28 x 28. 

The `mesh_shape` must be fixed before modeling and cannot be a free parameter of the model, because JAX uses the
mesh shape to define static shaped arrays which use the mesh to reconstruct the source. For a rectangular
mesh, the same number of pixels must be used in the y and x directions.

In [ ]:
mesh_pixels_yx = 28
mesh_shape = (mesh_pixels_yx, mesh_pixels_yx)

__Pixelization__

We create a `Pixelization` object to perform the pixelized galaxy reconstruction, which is made up of three
components:

- `mesh:` Different types of mesh can be used to perform the reconstruction, where the mesh changes the
details of how the reconstruction is performed (e.g. interpolation weights). In this example, we use a rectangular mesh.

- `regularization:` A pixelization uses many pixels to reconstruct the galaxy, which will often lead to over fitting
of the noise in the data and an unrealistically complex and structured reconstruction. Regularization smooths the
reconstruction solution by penalizing solutions where neighboring pixels have large flux differences.

In [ ]:
mesh = ag.mesh.RectangularUniform(shape=mesh_shape)
regularization = ag.reg.Constant(coefficient=1.0)

pixelization = ag.Pixelization(mesh=mesh, regularization=regularization)

__Fit__

This is to illustrate the API for performing a fit via a pixelization using standard objects like
the `Galaxy` and `FitInterferometer`.

We simply create a `Pixelization` and pass it to the galaxy, which then gets input into the fit.

In [ ]:
galaxy = ag.Galaxy(
    redshift=0.5,
    pixelization=pixelization,
)

galaxies = ag.Galaxies(galaxies=[galaxy])

fit = ag.FitInterferometer(
    dataset=dataset,
    galaxies=galaxies,
)

By plotting the fit, we see that the pixelized reconstruction does a good job at capturing the appearance of the galaxy
and fitting the data to roughly the noise level.

In [ ]:
fit_plotter = aplt.FitInterferometerPlotter(fit=fit)
fit_plotter.subplot_fit()
fit_plotter.subplot_fit_dirty_images()

Pixelizations have bespoke visualizations which show more details about the reconstruction, image-mesh
and other quantities.

These plots use an `InversionPlotter`, which gets its name from the internals of how pixelizations are performed in
the source code, where the linear algebra process which computes the pixel fluxes is called an inversion.

The `subplot_mappings` overlays colored circles in the image and reconstruction planes that map to one another.

In [ ]:
inversion_plotter = fit_plotter.inversion_plotter
inversion_plotter.subplot_of_mapper(mapper_index=0)
inversion_plotter.subplot_mappings(pixelization_index=0)

The inversion can be extracted directly from the fit the perform these plots, which we also use below
for various calculations

In [ ]:
inversion = fit.inversion

inversion_plotter = aplt.InversionPlotter(inversion=inversion)
inversion_plotter.subplot_of_mapper(mapper_index=0)

__Wrap Up__

Pixelizations are the most complex but also most powerful way to model a galaxy.

Whether you need to use them or not depends on the science you are doing. If you are only interested in measuring a
simple quantity, you can get away with using light profiles like a Sersic, MGE or shapelets to model a galaxy. Low
resolution data also means that using a pixelization is not necessary, as the complex structure of the galaxy is not
resolved anyway.

However, modeling complex galaxy light distributions requires this level of flexibility. Furthermore, if you are
interested in studying the properties of the galaxy itself, you won't find a better way to do this than using a
pixelization.

__Linear Objects__

An `Inversion` contains all of the linear objects used to reconstruct the data in its `linear_obj_list`.

This list may include the following objects:

 - `LightProfileLinearObjFuncList`: This object contains lists of linear light profiles and the functionality used
 by them to reconstruct data in an inversion. For example it may only contain a list with a single light profile
 (e.g. `lp_linear.Sersic`) or many light profiles combined in a `Basis` (e.g. `lp_basis.Basis`).

- `Mapper`: The linear objected used by a `Pixelization` to reconstruct data via an `Inversion`, where the `Mapper`
is specific to the `Pixelization`'s `Mesh`.

In this example, the only linear object used to fit the data was a `Pixelization`, thus the `linear_obj_list`
contains just one entry corresponding to a `Mapper`:

In [ ]:
print(inversion.linear_obj_list)

To extract results from an inversion many quantities will come in lists or require that we specific the linear object
we with to use.

Thus, knowing what linear objects are contained in the `linear_obj_list` and what indexes they correspond to
is important.

In [ ]:
print(f"Mapper = {inversion.linear_obj_list[0]}")

__Grids__

The role of a mapper is to map between the image-plane and reconstruction-plane.

This includes mapping grids corresponding to the data grid (e.g. the centers of each image-pixel in the image and
reconstruction plane) and the pixelization grid (e.g. the centre of the Delaunay triangulation in the image-plane and
reconstruction-plane).

All grids are available in a mapper via its `mapper` property.

In [ ]:
mapper = inversion.linear_obj_list[0]

# Centre of each masked image pixel in the image-plane.
print(mapper.mapper.image_plane_data_grid)

# Centre of each reconstruction pixel.
print(mapper.mapper.source_plane_data_grid)

# Centre of each pixelization pixel in the image-plane.
print(mapper.mapper.image_plane_mesh_grid)

# Centre of each pixelization pixel.
print(mapper.mapper.source_plane_mesh_grid)

__Reconstruction__

The reconstruction is also available as a 1D numpy array of values representative of the pixelization.

In [ ]:
print(inversion.reconstruction)

The (y,x) grid of coordinates associated with these values is given by the `Inversion`'s `Mapper`.

In [ ]:
mapper = inversion.linear_obj_list[0]
print(mapper.source_plane_mesh_grid)

The mapper also contains the (y,x) grid of coordinates that correspond to the image sub-pixels.

In [ ]:
print(mapper.source_plane_data_grid)

__Mapped Reconstructed Images__

The reconstruction(s) are mapped to the image-plane in order to fit the model.

These mapped reconstructed images are also accessible via the `Inversion`.

In [ ]:
print(inversion.mapped_reconstructed_operated_data.native)

__Linear Algebra Matrices (Advanced)__

To perform an `Inversion` a number of matrices are constructed which use linear algebra to perform the reconstruction.

These are accessible in the inversion object.

In [ ]:
print(inversion.curvature_matrix)
print(inversion.regularization_matrix)
print(inversion.curvature_reg_matrix)

__Evidence Terms (Advanced)__

In **HowToGalaxy** and the papers below, we cover how an `Inversion` uses a Bayesian evidence to quantify the goodness
of fit:

https://arxiv.org/abs/1708.07377
https://arxiv.org/abs/astro-ph/0601493

This evidence balances solutions which fit the data accurately, without using an overly complex regularization source.

The individual terms of the evidence and accessed via the following properties:

In [ ]:
print(inversion.regularization_term)
print(inversion.log_det_regularization_matrix_term)
print(inversion.log_det_curvature_reg_matrix_term)

__Simulated Interferometer__

We load the galaxy image from the pixelized inversion of a previous fit, which was performed on an irregular mesh.

Since irregular meshes cannot be directly used to simulate images, we interpolate the reconstruction onto a uniform
grid with shape `interpolated_pixelized_shape`. This grid should have a high resolution (e.g., 1000 × 1000) to preserve
all resolved structure from the original mesh.

In [ ]:
from scipy.interpolate import griddata

interpolation_grid = ag.Grid2D.uniform(shape_native=(200, 200), pixel_scales=0.05)

reconstruction = inversion.reconstruction
source_plane_mesh_grid = mapper.mapper.source_plane_mesh_grid

interpolated_reconstruction = griddata(
    points=source_plane_mesh_grid, values=reconstruction, xi=interpolation_grid
)

# As a pure 2D numpy array in case its useful for calculations
interpolated_reconstruction_ndarray = interpolated_reconstruction.reshape(
    interpolation_grid.shape_native
)

interpolated_reconstruction = ag.Array2D.no_mask(
    values=interpolated_reconstruction_ndarray,
    pixel_scales=interpolation_grid.pixel_scales,
)

To create the image, we evaluate image pixels and interpolate them onto the galaxy image.

This requires an image-plane grid of (y, x) coordinates. In this example, we use the real space mask grid.

To ensure accurate evaluation, we apply an 8×8 oversampling scheme.

In [ ]:
grid = ag.Grid2D.uniform(
    shape_native=real_space_mask.shape_native,
    pixel_scales=real_space_mask.pixel_scales,
    over_sample_size=8,
)

We create a galaxies object to generate the image onto which we overlay the interpolated galaxy image,
producing the interferometer image.

In [ ]:
galaxies = ag.Galaxies(
    galaxies=[
        galaxy,
    ]
)

Using the galaxies, we generate the galaxy image on the image-plane grid. This process incorporates
the `interpolated_reconstruction`, preserving the irregular and asymmetric morphological features captured by the reconstruction.

Next, we configure the grid and simulator settings to match the signal-to-noise ratio (S/N) and noise properties
of the observed data.

The `SimulatorInterferometer` takes the generated image and adds noise.

In [ ]:
simulator = ag.SimulatorInterferometer(
    uv_wavelengths=dataset.uv_wavelengths,
    exposure_time=300.0,
    noise_sigma=1000.0,
    transformer_class=ag.TransformerDFT,
)

dataset = simulator.via_image_from(image=interpolated_reconstruction)

plotter = aplt.InterferometerPlotter(dataset=dataset)

output = aplt.Output(path=".", filename="interpolated_reconstruction", format="png")

plotter = aplt.InterferometerPlotter(
    dataset=dataset, mat_plot_2d=aplt.MatPlot2D(output=output)
)

__Future Ideas / Contributions__

Here are a list of things I would like to add to this tutorial but haven't found the time. If you are interested
in having a go at adding them contact me on SLACK! :)

- Source gradient calculations.
- A calculation which shows differential effects across the reconstruction.